In [28]:
import cv2
import os
from IPython.display import clear_output
import numpy as np
from skimage import feature
from sklearn.svm import SVC
import joblib

FACE_CASCADE_PATH = 'haarcascade_frontalface_default.xml'
labels = ['Lelah', 'Sama-sama', 'Membaca', 'Apa', 'Hati-hati', 'Siapa', 'Makan', 'Nama', 'Halo', 'Bodoh', 'Maaf', 'Takut', 'Mau-Ingin', 'Saya', 'Anda', 'Berhenti', 'Terima kasih', 'Sombong', 'Cantik']

In [29]:


def draw_squares(image, name):
    img= cv2.cvtColor(cv2.medianBlur(image,7),cv2.COLOR_BGR2HSV)

    lower = np.array([0,100,0], dtype = "uint8")
    upper = np.array([20,255,255], dtype = "uint8")
    mask1 = cv2.inRange(img,lower,upper)

    lower = np.array([0, 30, 60], dtype = "uint8")
    upper = np.array([20, 150, 255], dtype = "uint8")
    mask2 = cv2.inRange(img,lower,upper)

    mask = cv2.bitwise_or(mask1, mask2)
    
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    length = len(contours)
    maxArea = -1
    if length > 0:
        ci = 0
        for i in range(length):
            temp = contours[i]
            area = cv2.contourArea(temp)
            if area > maxArea:
                maxArea = area
                ci = i
        res1 = contours[ci]
        maxArea = -1
        ci2 = -1
        for i in range(length):
            temp = contours[i]
            area = cv2.contourArea(temp)
            if area > maxArea and i != ci and area >= cv2.contourArea(res1) * 0.7:
                maxArea = area
                ci2 = i
        
        mask = np.zeros(mask.shape, np.uint8)
        cv2.fillPoly(mask, [res1], color=(255, 255, 255))
        if ci2 >= 0:
            cv2.fillPoly(mask, [contours[ci2]], color=(255, 255, 255))

    binary_mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)[1]
    mask = cv2.bitwise_and(img,img, mask = binary_mask)
    x, y, w, h = cv2.boundingRect(binary_mask)
    x = x - 10
    y = y - 10
    max_x = x+w+10
    max_y = y+h+10
    thickness = 3
    cv2.rectangle(image, (x - thickness + 1, y - 30), (max_x + thickness - 1, y), (0, 255, 0), -1)
    cv2.rectangle(image,(x, y),(max_x, max_y), (0, 255, 0), thickness)
    cv2.putText(image, name, (x + 8, y - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)

def getmask(img):
    img= cv2.cvtColor(cv2.medianBlur(img,7),cv2.COLOR_BGR2HSV)

    lower = np.array([0,100,0], dtype = "uint8")
    upper = np.array([20,255,255], dtype = "uint8")
    mask1 = cv2.inRange(img,lower,upper)

    lower = np.array([0, 30, 60], dtype = "uint8")
    upper = np.array([20, 150, 255], dtype = "uint8")
    mask2 = cv2.inRange(img,lower,upper)

    mask = cv2.bitwise_or(mask1, mask2)
    
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    length = len(contours)
    maxArea = -1
    if length > 0:
        ci = 0
        for i in range(length):
            temp = contours[i]
            area = cv2.contourArea(temp)
            if area > maxArea:
                maxArea = area
                ci = i
        res1 = contours[ci]
        maxArea = -1
        ci2 = -1
        for i in range(length):
            temp = contours[i]
            area = cv2.contourArea(temp)
            if area > maxArea and i != ci and area >= cv2.contourArea(res1) * 0.7:
                maxArea = area
                ci2 = i
        
        mask = np.zeros(mask.shape, np.uint8)
        cv2.fillPoly(mask, [res1], color=(255, 255, 255))
        if ci2 >= 0:
            cv2.fillPoly(mask, [contours[ci2]], color=(255, 255, 255))

    binary_mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)[1]
    mask = cv2.bitwise_and(img,img, mask = binary_mask)
    x_t, y_t, w_t, h_t = cv2.boundingRect(binary_mask)
    mask = mask[y_t:y_t+h_t, x_t:x_t+w_t]
    mask = cv2.resize(mask, (96, 96))
    return mask

def compute_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)
    gray_image = cv2.cvtColor(gray_image, cv2.COLOR_BGR2GRAY)
    hog_features, hogImage = feature.hog(gray_image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return hog_features

In [30]:
cap = cv2.VideoCapture(0)
SVM = joblib.load('SVC.pkl')
face_cascade = cv2.CascadeClassifier(FACE_CASCADE_PATH)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    bgr_frame = frame.copy()
    clear_output(wait=True)

    #removing face
    try:
        detected_faces = face_cascade.detectMultiScale(cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2GRAY), scaleFactor=1.2, minNeighbors=5, minSize=(35, 35))
        if len(detected_faces) > 0:
            for (x, y, w, h) in detected_faces: 
                bgr_frame[y:y+h, x:x+w] = 0
        f = [compute_hog_features(getmask(bgr_frame))]
        f = np.array(f)
        pred1 = SVM.predict(f)
        cv2.imshow('Removed Face', cv2.cvtColor(getmask(bgr_frame), cv2.COLOR_HSV2BGR))
        draw_squares(frame, labels[pred1[0]])
    except Exception as e:
        continue
    cv2.imshow('Full Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\wilbe\miniconda3\envs\computer_vision\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator SVC from version 1.6.0 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
